# SI 618: Data Manipulation and Analysis
## 12 - Big Data III: Spark DataFrames

### Dr. Chris Teplovs, School of Information, University of Michigan
<small><a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a>This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.

### [PDF of today's slides](resources/618_12_Spark_DataFrames.pdf) <-- LINK THERE

In [ ]:
df_from_other_list = spark.createDataFrame([('Chris',67),('Logan',70)], ['name','score'])
df_from_other_list.show()

In [ ]:
from pyspark.sql.types import FloatType
df_from_list = spark.createDataFrame([1.0,2.0,3.0,4.0,5.0], FloatType())
df_from_list.show()

In [ ]:
lot_rdd = sc.parallelize([('Chris',67),('Logan',70)])

In [ ]:
dfPeople = spark.createDataFrame(lot_rdd)
dfPeople.show()

In [ ]:
from pyspark.sql import Row
lot_rdd_named_columns = lot_rdd.map(lambda x: Row(name=x[0], score=int(x[1])))
dfPeople_named_columns = spark.createDataFrame(lot_rdd_named_columns)
dfPeople_named_columns.show()

In [ ]:
df = spark.read.json('s3://umsi-data-science/data/yelp/business.json')

In [ ]:
df.take(5)

In [ ]:
df.printSchema()

In [ ]:
df.select("name").show()

In [ ]:
df.select(df['name'], df['review_count'] + 1).show()

In [ ]:
df.filter(df['stars'] >= 4).show()

In [ ]:
df.groupBy('stars').count().show()

In [ ]:
df.groupBy('stars').count().sort('stars', ascending=False).show()

In [ ]:
df_from_other_list = spark.createDataFrame([('Chris',[67,42]),('Logan',[70,72])],['name','scores'])

In [ ]:
df_from_other_list.show()

In [ ]:
from pyspark.sql.functions import explode

In [ ]:
df_exploded = df_from_other_list.withColumn('score',explode('scores'))

In [ ]:
df_exploded.show()

In [ ]:
import pyspark.sql.functions as F
from pyspark.sql.functions import col
df_exploded.withColumn('good',F.when(df_exploded['score'] > 50,1).otherwise(0)).show()

In [ ]:
STOPWORDS = {'i', 'we', 'ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than'}
sw = sc.broadcast(STOPWORDS)

## Text analysis of Yelp reviews, warm-up

In [ ]:
# This takes about 3 minutes (180s) on a single-node m4.xlarge cluster
df = spark.read.json('s3://umsi-data-science/data/yelp/review.json.gz')

In [ ]:
df.printSchema()

### Q1: How many reviews are there?

In [ ]:
# Insert your code here

### Q2: How many positive (i.e. 4- or 5-star) reviews are there? 

In [ ]:
# Insert your code here

# BREAK


Let's read in two of the three data files from the Yelp academic dataset (https://www.kaggle.com/yelp-dataset/yelp-dataset) and examine the schemas for each one (we're skipping the reviews.json file for this part of the class):

In [ ]:
business = spark.read.json('s3://umsi-data-science/data/yelp/business.json')

In [ ]:
business.printSchema()

In [ ]:
# review = spark.read.json('s3://umsi-data-science/data/yelp/review.json.gz')

In [ ]:
# review.printSchema()

In [ ]:
tip = spark.read.json('s3://umsi-data-science/data/yelp/tip.json')

In [ ]:
tip.printSchema()

### Let's try to find the name of the business that has the highest number of "tips":

In [ ]:
most_tips = tip.groupBy('business_id').count().sort('count',ascending=False)

In [ ]:
from pyspark.sql.functions import col
most_tips = most_tips.withColumn('the_count',col('count'))

In [ ]:
most_tips.show()

In [ ]:
joined = most_tips.join(business,'business_id','left').sort('the_count',ascending=False)

In [ ]:
most_tips_joined = joined.select("name","the_count").filter(joined['the_count'] > 1000).collect()

In [ ]:
for b in most_tips_joined:
    print(b.name,b.count,b.the_count)


## Your turn
Use a combination of Spark and plain old python code to answer the following questions.  Include code and written responses in English for each question.

### Q3. How many businesses in the data set are located in the state of Ohio (OH)?

In [ ]:
business.filter(business.state == 'OH').count()

### Q4. How many Pennsylvania-based businesses have a hipster ambience?

In [ ]:
business.filter(business.state == 'PA').filter(business.attributes.Ambience.hipster == True).count()

### Q5. Which Nevada-based business has the most liked tip, and what is the text of the tip?

In [ ]:
tip.join(business,'business_id','inner').sort(tip.likes,ascending=False).select('name','likes','text','state').show()

### Q6. Excluding businesses in the state of Nevada, list 10 businesses with the highest number of tips

In [ ]:
business.filter(business.attributes.Ambience.hipster == True).filter(business.state != 'NV').count()

### Q7. List the names of the divey businesses from Ohio that have an overall rating of 4 or more stars and have at least 1000 tips.
You might want to do this in several steps.

In [ ]:
divey_counts = business.filter(business.attributes.Ambience.divey == True).filter(business.state == 'OH').filter(business.stars >= 4.0).join(tip,"business_id","right").groupBy('business_id').count()

In [ ]:
divey_counts = divey_counts.withColumn('the_count',col('count'))

In [ ]:
good_dives = divey_counts.filter(divey_counts.the_count >= 1000).join(business,'business_id').select('name').collect()

In [ ]:
for b in good_dives:
    print(b.name)

## END OF IN-CLASS NOTEBOOK
Please download the notebook in HTML and IPYNB formats and submit both to Canvas.

### **PLEASE REMEMBER TO TERMINATE YOUR CLUSTER(S) WHEN YOU ARE DONE!**